Proyek Pertama<br>
Kelas Machine Learning Terapan<br>
Ellion Blessan<br>
Sistem Rekomendasi Film Disney+

# Data Preparation

Mengimpor dependencies

In [1]:
import zipfile
import os
import shutil
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Dataset diambil dari Kaggle, yang berjudul [Disney+ Movies and TV Shows](https://www.kaggle.com/datasets/shivamb/disney-movies-and-tv-shows)

In [2]:
!pip install -q kaggle

from google.colab import files
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download -d shivamb/disney-movies-and-tv-shows

Saving kaggle.json to kaggle.json
100% 131k/131k [00:00<00:00, 589kB/s]
100% 131k/131k [00:00<00:00, 588kB/s]


Mengekstrak file zip yang telah diunduh

In [3]:
local_zip = 'disney-movies-and-tv-shows.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

Dataset yang didapat

In [4]:
df = pd.read_csv('disney_plus_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


# Data Exploration

Terdapat 1450 baris data film

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


Eksplorasi data, tiap film memiliki tipe, termasuk film layar lebar atau film tayangan TV, judul, sutradara, pemeran, negara, tanggal dimasukkan, tahun rilis, rating, durasi, kategori, dan deskripsi.

In [6]:
print('Jumlah data film: ', len(df['show_id'].unique()))
print('Jumlah data tipe film: ', len(df['type'].unique()))
print('Jumlah data sutradara film: ', len(df['director'].unique()))
print('Jumlah data pemeran film: ', len(df['cast'].unique()))
print('Jumlah data penilaian yang diberikan pengguna: ', len(df['rating'].unique()))
print('Jumlah data genre: ', len(df['listed_in'].unique()))

Jumlah data film:  1450
Jumlah data tipe film:  2
Jumlah data sutradara film:  610
Jumlah data pemeran film:  1194
Jumlah data penilaian yang diberikan pengguna:  10
Jumlah data genre:  329


Akan dilakukan rekomendasi berdasarkan genre

In [7]:
rec_df = df.loc[:, ['title', 'listed_in']]

Unique value yang terdapat pada genre

In [8]:
print('Banyak data: ', len(df['listed_in'].unique()))
print('Jenis Pembayaran yang diterima: ', df['listed_in'].unique())

Banyak data:  329
Jenis Pembayaran yang diterima:  ['Animation, Family' 'Comedy' 'Animation, Comedy, Family' 'Musical'
 'Docuseries, Historical, Music' 'Biographical, Documentary'
 'Action-Adventure, Superhero' 'Docuseries, Reality, Survival'
 'Animals & Nature, Docuseries, Family' 'Comedy, Family, Musical'
 'Documentary' 'Comedy, Family, Music' 'Documentary, Family'
 'Action-Adventure, Animals & Nature, Docuseries' 'Animals & Nature'
 'Animation' 'Animation, Kids' 'Comedy, Coming of Age, Drama'
 'Comedy, Family, Fantasy' 'Animation, Comedy, Drama'
 'Animation, Family, Fantasy' 'Action-Adventure, Animation, Comedy'
 'Comedy, Family' 'Action-Adventure, Comedy, Family' 'Lifestyle' 'Movies'
 'Action-Adventure, Science Fiction'
 'Action-Adventure, Fantasy, Superhero' 'Coming of Age, Music'
 'Animation, Drama' 'Concert Film, Music'
 'Animation, Comedy, Coming of Age' 'Animation, Comedy'
 'Animation, Crime, Family' 'Science Fiction' 'Action-Adventure, Fantasy'
 'Comedy, Fantasy, Kids' 'Actio

# Data Preprocessing

Terlihat bahwa data genre terdiri dari 1-3 genre dalam satu baris. Akan dilakukan pembersihan sehingga data menampilkan 1 genre pertama saja.

In [9]:
film_df = df.loc[:, ['title', 'listed_in']]
film_df.head()

,title,listed_in
0,Duck the Halls: A Mickey Mouse Christmas Special,"Animation, Family"
1,Ernest Saves Christmas,Comedy
2,Ice Age: A Mammoth Christmas,"Animation, Comedy, Family"
3,The Queen Family Singalong,Musical
4,The Beatles: Get Back,"Docuseries, Historical, Music"


Menggunakan str.split untuk memisahkan data genre dengan delimiter ', '

In [10]:
film_df = pd.concat([film_df, film_df['listed_in'].str.split(', ', expand=True)], axis=1)
film_df.head()

,title,listed_in,0,1,2
0,Duck the Halls: A Mickey Mouse Christmas Special,"Animation, Family",Animation,Family,None
1,Ernest Saves Christmas,Comedy,Comedy,None,None
2,Ice Age: A Mammoth Christmas,"Animation, Comedy, Family",Animation,Comedy,Family
3,The Queen Family Singalong,Musical,Musical,None,None
4,The Beatles: Get Back,"Docuseries, Historical, Music",Docuseries,Historical,Music


Hanya digunakan genre pertama untuk merekomendasikan film

In [11]:
genre_df = film_df.loc[:, ['title', 0]]
genre_df.rename(columns={0: 'genre'}, inplace=True)
genre_df.head()

,title,genre
0,Duck the Halls: A Mickey Mouse Christmas Special,Animation
1,Ernest Saves Christmas,Comedy
2,Ice Age: A Mammoth Christmas,Animation
3,The Queen Family Singalong,Musical
4,The Beatles: Get Back,Docuseries


Memeriksa ada/tidaknya null value pada data

In [12]:
genre_df.isnull().sum()

title    0
genre    0
dtype: int64

Data genre sudah menjadi 1 genre per baris. Namun terdapat genre yang berupa klausa maupun 2 kata yang dihubungkan.

In [13]:
genre_df.genre.unique()

array(['Animation', 'Comedy', 'Musical', 'Docuseries', 'Biographical',
       'Action-Adventure', 'Animals & Nature', 'Documentary', 'Lifestyle',
       'Movies', 'Coming of Age', 'Concert Film', 'Science Fiction',
       'Drama', 'Anthology', 'Reality', 'Family', 'Kids', 'Crime',
       'Talk Show', 'Fantasy', 'Series', 'Buddy',
       'Game Show / Competition', 'Variety', 'Dance', 'Historical',
       'Sports', 'Music'], dtype=object)

Dilakukan replace data genre yang berupa klausa supaya dapat diproses dengan benar

In [14]:
genre_df.genre = genre_df.genre.replace(
    ['Action-Adventure', 'Animals & Nature', 'Coming of Age', 'Concert Film', 'Science Fiction', 'Talk Show', 'Game Show / Competition'], 
    ['action_adventure', 'animals_nature', 'coming_of_age', 'concert_film', 'sci_fi', 'talk_show', 'game_show'])

In [15]:
data = genre_df
data.sample(5)

,title,genre
777,Cars 2,Animation
22,Get a Horse!,action_adventure
293,Secrets of Sulphur Springs,Family
60,Lost Cities with Albert Lin: The Great Flood,Documentary
782,Cars Toon: Mater Private Eye,Animation


# Modeling

Menggunakan TfidfVectorizer untuk mengekstrak fitur dalam teks genre

In [16]:
vect = TfidfVectorizer()
vect.fit(data['genre'])
vect.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['action_adventure',
 'animals_nature',
 'animation',
 'anthology',
 'biographical',
 'buddy',
 'comedy',
 'coming_of_age',
 'concert_film',
 'crime',
 'dance',
 'documentary',
 'docuseries',
 'drama',
 'family',
 'fantasy',
 'game_show',
 'historical',
 'kids',
 'lifestyle',
 'movies',
 'music',
 'musical',
 'reality',
 'sci_fi',
 'series',
 'sports',
 'talk_show',
 'variety']

Jumlah baris data dan jumlah genre

In [17]:
vect_matrix = vect.fit_transform(data['genre']) 
vect_matrix.shape 

(1450, 29)

Tampilan matrix TF-IDF

In [18]:
pd.DataFrame(
    vect_matrix.todense(), 
    columns=vect.get_feature_names(),
    index=data.title
)

,action_adventure,animals_nature,animation,anthology,biographical,buddy,comedy,coming_of_age,concert_film,crime,...,lifestyle,movies,music,musical,reality,sci_fi,series,sports,talk_show,variety
title,,,,,,,,,,,,,,,,,,,,,
Duck the Halls: A Mickey Mouse Christmas Special,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ernest Saves Christmas,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ice Age: A Mammoth Christmas,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Queen Family Singalong,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
The Beatles: Get Back,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X-Men Origins: Wolverine,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Night at the Museum: Battle of the Smithsonian,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Eddie the Eagle,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Menggunakan Cosine similarity untuk membandingkan kemiripan antar data

In [19]:
cos = cosine_similarity(vect_matrix)
cos_sim_df = pd.DataFrame(cos, index=data['title'], columns=data['title'])
print('Shape:', cos_sim_df.shape)
cos_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (1450, 1450)


title,DuckTales,Remember the Titans,LEGO Star Wars: The New Yoda Chronicles - Clash of the Skywalkers,Disney Channel Games 2008,Spider-Woman
title,,,,,
Kickin' It,1.0,0.0,1.0,0.0,1.0
Mickey Mouse Club (1955-59),0.0,0.0,0.0,0.0,0.0
Disney Amphibia,0.0,0.0,0.0,0.0,0.0
Playing with Sharks,0.0,0.0,0.0,0.0,0.0
G-Force,1.0,0.0,1.0,0.0,1.0
Return from Witch Mountain,1.0,0.0,1.0,0.0,1.0
The Strongest Man in the World,0.0,0.0,0.0,0.0,0.0
Pirates of the Caribbean: Dead Men Tell No Tales,1.0,0.0,1.0,0.0,1.0
Marvel Rising: Heart of Iron,1.0,0.0,1.0,0.0,1.0


Fungsi untuk memberikan rekomendasi film berdasarkan genre

In [20]:
def film_recommendations(nama_film, similarity_data=cos_sim_df, items=data[['title', 'genre']], k=5):
    index = similarity_data.loc[:,nama_film].to_numpy().argpartition(
        range(-1, -k, -1))
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    closest = closest.drop(nama_film, errors='ignore')
 
    return pd.DataFrame(closest).merge(items).head(k)

Rekomendasi film yang mirip dengan Moana

In [21]:
film_recommendations("Moana")

,title,genre
0,Captain Sparky vs. The Flying Saucers,action_adventure
1,Minutemen,action_adventure
2,Baby's Day Out,action_adventure
3,Miracle Landing on the Hudson,action_adventure
4,DuckTales,action_adventure


# Evaluation

Sistem rekomendasi dievaluasi dengan menghitung precision. Didapatkan dengan menghitung jumlah rekomendasi yang relevan dibagi dengan jumlah rekomendasi

In [25]:
def eval_rec(nama_film, similarity_data=cos_sim_df, items=data[['title', 'genre']], k=5):
    index = similarity_data.loc[:,nama_film].to_numpy().argpartition(
        range(-1, -k, -1))
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    closest = closest.drop(nama_film, errors='ignore')
    top = pd.DataFrame(closest).merge(items).head(k)
    req = data[data.title.eq(nama_film)]
    req_genre = req.genre
    top_genre = top.genre
    true = 0
    for genres in top_genre:
      if genres == req_genre.iloc[0]:
        true += 1
    precision = true / len(top)
    return precision

Menghitung nilai precision untuk semua data dan dirata-rata. Didapatkan precision sebesar 99%

In [26]:
all_precision = []
for title in data['title']:
  precision = eval_rec(title)
  all_precision.append(precision)
mean_precision = sum(all_precision) / len(all_precision)
print('Precision sistem rekomendasi sebesar', "{:.0%}".format(mean_precision))

Precision sistem rekomendasi sebesar 99%
